<a href="https://colab.research.google.com/github/badicev/KaggleCode/blob/main/Global_AI_Hub_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>